In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_chinese import ChatZhipuAI, ZhipuAIEmbeddings
from langchain_chinese import (
    LocalFilesLoader,
    create_qa_chain,
    create_qa_toolkits,
    create_reason_agent,
)
from langchain_community.vectorstores import Chroma

docs = LocalFilesLoader()
chunks = docs.load_and_split()

db = Chroma.from_documents(chunks, ZhipuAIEmbeddings())

chain = create_qa_chain(ChatZhipuAI(), db.as_retriever())

In [3]:
chain.invoke("供应商达标的业绩要求是什么")

AIMessage(content='供应商达标的业绩要求主要包括：\n\n1. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。\n2. 稳定的财务状况：供应商应具备健全和稳定的财务状况，以及良好的信用记录。\n3. 流动资金：供应商应具备足够的流动资金来支持合同执行。\n4. 行业经验：供应商应具有至少五年的相关行业经验。\n5. 业务声誉：供应商应有良好的业务声誉和客户满意度记录。\n\n以上这些要求旨在保证供应商具备良好的经营能力、稳定的财务状况和足够的行业经验，以确保双方合作的质量和效率。')

In [8]:
for chunk in chain.stream("请从资料中确认供应商达标的标准是什么"):
    # print(type(chunk))
    print(chunk)

content='根据'
content='提供的'
content='资料'
content='，'
content='供应商'
content='达标'
content='的标准'
content='如下'
content='：\n\n1'
content='.'
content=' 基'
content='本'
content='资质'
content='要求'
content='：\n  '
content=' -'
content=' 具'
content='备'
content='合法'
content='有效的'
content='营业执照'
content='。\n  '
content=' -'
content=' 拥'
content='有'
content='税务'
content='登记'
content='证'
content='，'
content='并'
content='符合'
content='国家'
content='税收'
content='法律法规'
content='。\n  '
content=' -'
content=' 拥'
content='有'
content='有效的'
content='组织'
content='机构'
content='代码'
content='证'
content='。\n\n2'
content='.'
content=' 经营'
content='和'
content='财务'
content='能力'
content='：\n  '
content=' -'
content=' 与'
content='公司'
content='合作的'
content='月份'
content='，'
content='经'
content='公司'
content='销售'
content='的产品'
content='月'
content='总价'
content='不得'
content='低于'
content='人民币'
content='3'
content='万元'
content='。\n  '
content=' -'
content=' 具'
content='备'
content='健全'
content='和'
content='稳定的'
content='财务'
conten

In [4]:
async for chunk in chain.astream_events("供应商达标的业绩要求是什么?", version="v1"):
	# print(chunk)
	# print(chunk['event'])
	if(chunk['event'] == "on_chat_model_stream"):
		print(chunk['data']['chunk'].content, end="|", flush=True)
	elif(chunk['event'] == "on_retriever_end"):
		for doc in chunk['data']['output']['documents']:
			print(doc.metadata['source'])


/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/求职简历.docx
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
供应商|达|标的|业绩|要求|主要包括|：

1|.| 月|销售额|：|供应商|需|保证|在与|本公司|合作的|月份|中|，|经|我|方|销售|的产品|月|总价|不得|低于|人民币|3|万元|。

2|.| 稳|定的|财务|状况|：|供应商|应|具备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。

3|.| 行|业|经验|：|供应商|应|具有|至少|五|年的|相关|行业|经验|。

4|.| 业务|声誉|：|供应商|应有|良好的|业务|声誉|和|客户|满意度|记录|。

这些|业绩|要求|旨在|确保|供应商|具备|良好的|经营|能力|、|稳定的|财务|状况|和|丰富的|行业|经验|，|以满足|合作|的需要|。||

In [4]:
llm_zhipu = ChatZhipuAI(model="glm-4")
tools = create_qa_toolkits(qa_chain=chain)
agent = create_reason_agent(llm_zhipu, tools=tools)

In [6]:
from langchain.tools.render import render_text_description, render_text_description_and_args
print(render_text_description_and_args(tools))
print(render_text_description(tools))

ask_document: 根据资料库回答问题。考虑上下文信息，确保问题对相关概念的定义表述完整。
    args:
    - query 类型是str, 用户问题的文字描述
    , args: {'query': {'title': '提问内容', 'description': '用户问题的文字描述，必须是字符串', 'type': 'string'}}
ask_document: 根据资料库回答问题。考虑上下文信息，确保问题对相关概念的定义表述完整。
    args:
    - query 类型是str, 用户问题的文字描述
    


In [8]:
tools[0].invoke("供应商怎样才算合格？")

AIMessage(content='供应商要算合格，需要满足以下条件：\n\n1. **基本资质要求**：\n   - 拥有合法有效的营业执照。\n   - 具备税务登记证，并符合国家税收法律法规。\n   - 拥有有效的组织机构代码证。\n\n2. **经营和财务能力**：\n   - 在与公司合作的月份中，经公司销售的产品月总价不得低于人民币3万元。\n   - 具备健全和稳定的财务状况，以及良好的信用记录。\n   - 拥有足够的流动资金来支持合同执行。\n\n3. **产品和服务质量**：\n   - 供应的产品必须符合国家及行业标准，并通过相关质量认证。\n   - 拥有高标准的服务体系，并承诺在合作期间提供持续、稳定、优质的服务。\n\n4. **行业经验和声誉**：\n   - 具有至少五年的相关行业经验。\n   - 拥有良好的业务声誉和客户满意度记录。\n\n5. **环境保护和社会责任**：\n   - 遵循环境保护法律法规，在经营活动中实施环保措施。\n   - 积极承担社会责任，遵守劳工权益法律法规，并推行公平、平等、安全和健康的劳动环境。\n\n6. **申请和审核流程**：\n   - 按规定提交完整的申请材料。\n   - 通过公司的初步审核、现场考察及终审决定。\n\n只有当供应商满足上述所有条件，才能被认为是合格的供应商，并有机会参与投标。')

In [6]:
await tools[0].ainvoke("供应商怎样才算合格？")

"content='供应商合格的标准如下：\\n\\n一、基本资质要求\\n1. 具备合法有效的营业执照。\\n2. 具备税务登记证，并符合国家税收法律法规。\\n3. 拥有有效的组织机构代码证。\\n\\n二、经营和财务能力\\n1. 具备健全和稳定的财务状况，以及良好的信用记录。\\n2. 拥有足够的流动资金来支持合同执行。\\n3. 合作的月份，经我方销售的产品月总价不得低于人民币3万元。\\n\\n三、产品和服务质量\\n1. 提供的产品必须符合国家及行业标准，并通过相关质量认证。\\n2. 具备高标准的服务体系，并承诺在合作期间提供持续、稳定、优质的服务。\\n\\n四、行业经验和声誉\\n1. 具有至少五年的相关行业经验。\\n2. 有良好的业务声誉和客户满意度记录。\\n\\n五、社会责任和可持续性\\n1. 应遵循环境保护法律法规，并在经营活动中实施环保措施。\\n2. 积极承担社会责任，遵守劳工权益法律法规，并推行公平、平等、安全和健康的劳动环境。\\n\\n只有当供应商满足以上所有标准，才能被认为是合格的供应商。'"

In [5]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [8]:
for chunk in agent_executor.stream({"input": "供应商怎样才算合格？"}):
    print(chunk)

{'actions': [AgentAction(tool='ask_document', tool_input={'query': '供应商评价体系和合格分数线是什么？'}, log='### 关键概念:\n- 供应商合格标准\n\n### 概念拆解:\n- 供应商合格标准 - \n  - 供应商评价体系\n  - 合格分数线\n  - 评价项目\n\n### 反思:\n- 需要了解是否有现成的供应商评价体系。\n- 要确定是否已有合格分数线或评价项目的具体信息。\n\n### 思考:\n- 我需要先了解整个供应商评价体系，才能确定合格的标准。\n- 评价体系和合格分数线可能来源于相同或不同的数据源，尝试一次性获取。\n- 如果没有直接的数据源，我可能需要拆解评价体系，单独查询各个评价项目。\n\n### 推理:\n- 通过资料库查询“供应商评价体系”及“合格分数线”的相关信息。\n\n### 计划:\n- 使用 `ask_document` 工具查询“供应商评价体系”及“合格分数线”。\n\n### 输出所选择执行的动作/工具/指令:\n```json\n{"name": "ask_document", "args": {"query": "供应商评价体系和合格分数线是什么？"}}\n```')], 'messages': [AIMessage(content='### 关键概念:\n- 供应商合格标准\n\n### 概念拆解:\n- 供应商合格标准 - \n  - 供应商评价体系\n  - 合格分数线\n  - 评价项目\n\n### 反思:\n- 需要了解是否有现成的供应商评价体系。\n- 要确定是否已有合格分数线或评价项目的具体信息。\n\n### 思考:\n- 我需要先了解整个供应商评价体系，才能确定合格的标准。\n- 评价体系和合格分数线可能来源于相同或不同的数据源，尝试一次性获取。\n- 如果没有直接的数据源，我可能需要拆解评价体系，单独查询各个评价项目。\n\n### 推理:\n- 通过资料库查询“供应商评价体系”及“合格分数线”的相关信息。\n\n### 计划:\n- 使用 `ask_document` 工具查询“供应商评价体系”及“合格分数线”。\n\n### 输出所选择执行的动作/工具/指令:\n```json\n{"name": "ask_

In [10]:
async for chunk in agent_executor.astream_events({"input": "供应商怎样才算合格？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="", flush=True)
    elif(event == "on_chain_end"):
        print(event)
        print(chunk['data']['output'])
    else:
        print(event)

on_chain_start
on_chain_start
on_chain_start
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_stream
on_chain_stream
on_chain_end

on_chain_end
{'agent_scratchpad': ''}
on_chain_end
{'input': '供应商怎样才算合格？', 'intermediate_steps': [], 'agent_scratchpad': ''}
on_prompt_start
on_prompt_end
on_chat_model_start
### 关键概念:
- 供应商合格标准

### 概念拆解:
- 供应商合格标准 - 
  - 供应商评价体系
  - 合格分数线
  - 评价具体指标

### 反思:
- 需要了解是否有现成的供应商评价体系。
- 要确定是否已经有具体的合格分数线和评价指标。

### 思考:
- 首先，需要了解供应商评价体系是否存在，如果存在，需要获取这个体系的具体内容。
- 其次，要明确合格的标准，即合格分数线和评价的具体指标。

### 推理:
- 可以通过查询资料库来获取有关供应商评价体系的信息。

### 计划:
- 使用 `ask_document` 工具查询供应商评价体系的相关信息。

### 输出所选择执行的动作/工具/指令:
```json
{"name": "ask_document", "args": {"query": "供应商评价体系包括哪些标准和指标？"}}
```on_chat_model_end
on_parser_start
on_parser_end
on_chain_stream
on_chain_end
tool='ask_document' tool_input={'query': '供应商评价体系包括哪些标准和指标？'} log='### 关键概念:\n- 供应商合格标准\n\n### 概念拆解:\n- 供应商合格标准 - \n  - 供应商评价体系\n  - 合格分数线\n  - 评价具体指标\n\n### 反思:\n- 需要了解是否有现成的供应商评价体系。\n- 要确定是否已经有具体的合格